In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models

from nbdt.model import SoftNBDT
from nbdt.models import wrn28_10_cifar10
from decompose import decompose

import numpy as np
import random
import matplotlib.pyplot as plt

from data_loader_cifar10 import get_train_valid_loader, get_test_loader
from pytorchtools import EarlyStopping, test_model, train_model, train_model_distil

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

stty: 'standard input': Inappropriate ioctl for device


not enough values to unpack (expected 2, got 0)


# Load Data

In [2]:
batch_size = 128

trainloader, validloader = get_train_valid_loader("./data", batch_size, False, random_seed=0)
testloader = get_test_loader("./data", batch_size, random_seed=0)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


# Pretrained (WideResNet)

### Test Pretrained model

In [9]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
)

nParam = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parameters in model: {nParam}")

acc, carbontracker_full = test_model(model, testloader, resnet=True)
acc

Parameters in model: 36479194
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA G

0.9732

## Decomp pretrained model and test

In [ ]:
# Original: time=10, acc=0.975, energy=0.528, params=36479194
# Factor 2 : time=7, acc=0.974, energy=0.177, params=20540634
# Factor 4 : time=5, acc=0.971, energy=0.131, params=10416474
# Factor 6 : time=5, acc=0.965, energy=0.133, params=6993818
# Factor 8 : time=5, acc=0.957, energy=0.141, params=5354394
# Factor 10: time=5, acc=0.951, energy=0.160, params=4341978


# OLD DATA:
# Original: time=10, acc=0.975, energy=0.346, params=36479194
# Factor 2 : time=7, acc=0.974, energy=0.256, params=20540634
# Factor 4 : time=5, acc=0.971, energy=0.197, params=10416474
# Factor 6 : time=5, acc=0.965, energy=0.195, params=6993818
# Factor 8 : time=5, acc=0.957, energy=0.195, params=5354394
# Factor 10: time=5, acc=0.951, energy=0.255, params=4341978

In [3]:
#for factor in [2, 4, 6, 8, 10]:
for factor in [20]:
    model = wrn28_10_cifar10()
    model = SoftNBDT(
        pretrained=True,
        dataset='CIFAR10',
        arch='wrn28_10_cifar10',
        model=model
    ).to(device)

    factor = factor
    model = decompose(model, factor, './Models/', include_params_and_factor=True, ignore_list=['init_block', 'identity_conv'])

    nParam = sum(p.numel() for p in model.parameters() if p.requires_grad)

    n_epochs = 5
    patience = 15
    lr = 0.004
    save_path = f'./Results/Distil/WRN_from_pretrained_{factor=}_{nParam=}.pt'

    criterion = torch.nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

    train_model(model, trainloader, validloader, optimizer, criterion, n_epochs, patience, save_path, resnet=True)
    test_model(model, testloader, resnet=True)
    print(f'{factor=}')

/home/borge/anaconda3/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


2317146
[1/5] train_loss: 1.66746 valid_loss: 1.61400
Validation loss decreased (inf --> 1.614000).  Saving model ...
[2/5] train_loss: 1.57869 valid_loss: 1.58619
Validation loss decreased (1.614000 --> 1.586185).  Saving model ...
[3/5] train_loss: 1.54618 valid_loss: 1.57269
Validation loss decreased (1.586185 --> 1.572686).  Saving model ...
[4/5] train_loss: 1.52820 valid_loss: 1.57223
Validation loss decreased (1.572686 --> 1.572234).  Saving model ...
[5/5] train_loss: 1.51532 valid_loss: 1.55739
Validation loss decreased (1.572234 --> 1.557385).  Saving model ...
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: INFO - Traceback (most recent call last):
  File "/home/borge/anaconda3/lib/python3.9/site-packages/carbontracker/emissions/intensity/intensity.py", line 62, in carbon_intensity
    carbon_intensity = fetcher.carbon_intensity(g_location, time_dur)
  File "/home/borge/anaconda3/lib/python3.9/site-packages/ca

## Decompose and distil model

In [ ]:
# Distillation results:
# lr=0.005
# Factor 10: Params=4341978, accuracy=, Wh=, CO2=

In [11]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
).to(device)

factor = 8
model = decompose(model, factor, './Models/', include_params_and_factor=True, ignore_list=['init_block', 'identity_conv'])

teacher_model = wrn28_10_cifar10()
teacher_model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=teacher_model
).to(device)

n_epochs = 5
patience = 15
#lr = 0.004
lr = 0.008
save_path = f'./Results/Distil/test_decomp_{factor=}.pt'

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5e-4)

train_model_distil(model, trainloader, validloader, teacher_model, optimizer, criterion, n_epochs, patience, save_path, resnet=True)
test_model(model, testloader, resnet=True)

/home/borge/anaconda3/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


2317146
[0/5] train_loss: 1.66166 valid_loss: 1.61825
Validation loss decreased (inf --> 1.618246).  Saving model ...


KeyboardInterrupt: 

# Model architecture

In [3]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
).to(device)

factor = 4
decompose(model, factor, './Models/', include_params_and_factor=True)

/home/borge/anaconda3/lib/python3.9/site-packages/tensorly/backend/core.py:1106: UserWarning: In partial_svd: converting to NumPy. Check SVD_FUNS for available alternatives if you want to avoid this.
  warnings.warn('In partial_svd: converting to NumPy.'


10416474


SoftNBDT(
  (model): CIFARWRN(
    (features): Sequential(
      (init_block): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (stage1): Sequential(
        (unit1): PreResUnit(
          (body): PreResBlock(
            (conv1): PreConvBlock(
              (bn): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ): ReLU(inplace=True)
              (conv): Sequential(
                (0): Conv2d(16, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): Conv2d(40, 160, kernel_size=(1, 1), stride=(1, 1))
              )
            )
            (conv2): PreConvBlock(
              (bn): BatchNorm2d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (activ): ReLU(inplace=True)
              (conv): Sequential(
                (0): Conv2d(160, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
                (1): Conv2d(40, 

In [10]:
model = wrn28_10_cifar10()
model = SoftNBDT(
    pretrained=True,
    dataset='CIFAR10',
    arch='wrn28_10_cifar10',
    model=model
).to(device)

nParam = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"Parameters in model: {nParam}")

Parameters in model: 36479194


In [6]:
from tensorly.decomposition import parafac, partial_tucker, matrix_product_state
import tensorly as tl

layer = nn.Conv2d(160, 100, 3, 1)

data = layer.weight.data
data2D = tl.base.unfold(data, 0)

first, last = matrix_product_state(data2D, rank=16)

TypeError: transpose() received an invalid combination of arguments - got (list), but expected one of:
 * (int dim0, int dim1)
 * (name dim0, name dim1)


# Final test of models

In [7]:
for factor, path in zip(
    [2, 4, 6, 8, 10],
    [
        "WRN_from_pretrained_factor=2_nParam=20540634.pt",
        "WRN_from_pretrained_factor=4_nParam=10416474.pt",
        "WRN_from_pretrained_factor=6_nParam=6993818.pt",
        "WRN_from_pretrained_factor=8_nParam=5354394.pt",
        "WRN_from_pretrained_factor=10_nParam=4341978.pt",
    ],
):
    model = wrn28_10_cifar10()
    model = SoftNBDT(
        pretrained=True,
        dataset='CIFAR10',
        arch='wrn28_10_cifar10',
        model=model
    ).to(device)

    factor = factor
    model = decompose(model, factor, './Models/', include_params_and_factor=True, ignore_list=['init_block', 'identity_conv'])
    model.load_state_dict(torch.load('./Results/Distil/' + path))

    test_model(model, testloader, resnet=True)
    print(f'{factor=}')

20540634
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
C

In [ ]:
for factor, path in zip(
    [20],
    [
        'WRN_from_pretrained_factor=20_nParam=2317146.pt'
    ],
):
    model = wrn28_10_cifar10()
    model = SoftNBDT(
        pretrained=True,
        dataset='CIFAR10',
        arch='wrn28_10_cifar10',
        model=model
    ).to(device)

    factor = factor
    model = decompose(model, factor, './Models/', include_params_and_factor=True, ignore_list=['init_block', 'identity_conv'])
    model.load_state_dict(torch.load('./Results/Distil/' + path))

    test_model(model, testloader, resnet=True)
    print(f'{factor=}')

20540634
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
CarbonTracker: The following components were found: GPU with device(s) NVIDIA GeForce RTX 3060 Ti.
C